# Generate list of itemID for each events
Currently we are focusing on 6 events.
- inputevents
- outputevents
- chartevents
- labevents
- microbiologyevents
- prescriptions

For each one of these events, in this step, we generate the list of itemID that used by metavision system.

In [2]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [3]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load metavision admission ID

In [4]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

# select outputevent itemid
We select the itemID from outputevent that id is more than 200000.
ID numbers that are more than 200000 is from metavision system.

In [5]:
sql = 'select distinct itemid from mimiciii.outputevents where itemid >= 200000'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
output_itemid = []
output_itemid_txt = ""

for r in res:
    output_itemid.append(r[0])

print "len(output_itemid) = "+ str(len(output_itemid))

len(output_itemid) = 68


# select inputevent itemid
We select the itemID from inputevents that id is more than 200000.
ID numbers that are more than 200000 is from metavision system.

In [6]:
sql = 'select distinct itemid from mimiciii.inputevents_mv where itemid >= 200000'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
input_itemid = []
input_itemid_txt = ""

for r in res:
    input_itemid.append(r[0]);

print "len(input_itemid) = "+ str(len(input_itemid))

len(input_itemid) = 278


# select inputevent itemid
We select the itemID from chartevent that id is more than 200000.
ID numbers that are more than 200000 is from metavision system.

In [7]:
sql = 'select distinct itemid from mimiciii.chartevents where itemid >= 200000'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
chart_itemid = []
chart_itemid_txt = ""

for r in res:
    chart_itemid.append(r[0]);

print "len(chart_itemid) = "+ str(len(chart_itemid))

len(chart_itemid) = 661


# select labevent itemid
In this step, we select all the itemid that appears in labevent table.

In [8]:
sql = 'select distinct itemid from mimiciii.labevents'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
print 'len(res) = ' +str(len(res));

numberOfAdmissionThatUseItemid = []
i=0
for r in res:
    r=r[0]
    sql = 'select count(distinct hadm_id) from mimiciii.labevents where itemid='+str(r)+' AND hadm_id in(' + admission_ids_txt + ')'
    cur = conn.cursor()
    cur.execute(sql)
    r2 = cur.fetchall()
    numberOfAdmissionThatUseItemid.append((r, r2[0],))
    print '.',
    if(i%1000==0): print i
    i+=1

numberOfAdmissionThatUseItemid = sorted(numberOfAdmissionThatUseItemid, key=lambda tup: tup[1]);
numberOfAdmissionThatUseItemid.reverse()

lab_itemid = []
for r in numberOfAdmissionThatUseItemid:
    lab_itemid.append(r[0])

len(res) = 729
. 0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

KeyboardInterrupt: 

# Labevent histogram
- y axis is the number of distinct admission id that use a particular labitemID.
- x axis: each position in x axis correspond to a particular labitemID.
- For example there are around 20 labitemsID that is used by all admissions.
- From this graph we can also conclude that only top 100 labitemid is observed in most admissions. The rest of the labitemID is very rarely used.

In [ ]:
plt.figure(figsize=(10,5))
plt.bar([i for i in range(len(numberOfAdmissionThatUseItemid))],[int(r[1][0]) for r in numberOfAdmissionThatUseItemid])

# select microbiologyevents itemid
This table give the microbiology test result of patients.
Each row contains 3 id and test result
- specimen type (what is the specimen to test, e.g. urine, blood, etc.)
- organism type (what is the organism to test, e.g. bacteria, yeast, ENTEROCOCCUS SP.)
- antibiotic type (? i.e. LEVOFLOXACIN)

Here in this step, we extract every combination of 3 ids that appear in dataset.

In [9]:
sql = 'select distinct (spec_itemid,org_itemid,ab_itemid),spec_itemid,org_itemid,ab_itemid from mimiciii.microbiologyevents'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
print 'len(res) = ' +str(len(res));

numberOfAdmissionThatUseItemid = []
i=0
for r in res:
    sql = 'select count(distinct hadm_id) from mimiciii.microbiologyevents where hadm_id in(' + admission_ids_txt + ')'
    if(r[1]!=None): sql += ' and spec_itemid=' + str(r[1])
    else: sql += ' and spec_itemid is null'
    if(r[2]!=None): sql += ' and org_itemid=' + str(r[2])
    else: sql += ' and org_itemid is null'
    if(r[3]!=None): sql += ' and ab_itemid=' + str(r[3])
    else: sql += ' and ab_itemid is null'
    
    cur = conn.cursor()
    cur.execute(sql)
    r2 = cur.fetchall()
    numberOfAdmissionThatUseItemid.append((r[0], r2[0],))
    print '.',
    if(i%1000==0): print i
    i+=1

numberOfAdmissionThatUseItemid = sorted(numberOfAdmissionThatUseItemid, key=lambda tup: tup[1]);
numberOfAdmissionThatUseItemid.reverse()

microbio_itemid = []
for r in numberOfAdmissionThatUseItemid:
    microbio_itemid.append(r[0])

 len(res) = 9085
. 0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

# microbiology histogram
- y axis : number of distinct admissions that has a particular microbiology test.
- x axis : a particular test, identified by (spec_id, org_id, ab_id)

In [ ]:
plt.figure(figsize=(10,5))
plt.bar([i for i in range(len(numberOfAdmissionThatUseItemid))],[int(r[1][0]) for r in numberOfAdmissionThatUseItemid])

# select prescription formulary_drug_cd
Formulary_drug_cd is a medicine type. Each record in this tables gives a dose of medicine that is given to patient at a particular time.

In this step, we select all the formulary_drug_cd that appears in prescriptions table.


In [10]:
sql = 'select distinct formulary_drug_cd from mimiciii.prescriptions'
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
print 'len(res) = ' +str(len(res));

numberOfAdmissionThatUseItemid = []
i=0
for r in res:
    sql = 'select count(distinct hadm_id) from mimiciii.prescriptions where hadm_id in(' + admission_ids_txt + ') '
    sql += 'and formulary_drug_cd = \'' + r[0] + '\''
    
    cur = conn.cursor()
    cur.execute(sql)
    r2 = cur.fetchall()
    numberOfAdmissionThatUseItemid.append((r[0], r2[0],))
    print '.',
    if(i%1000==0): print i
    i+=1
    
numberOfAdmissionThatUseItemid = sorted(numberOfAdmissionThatUseItemid, key=lambda tup: tup[1]);
numberOfAdmissionThatUseItemid.reverse()

prescript_itemid = []
for r in numberOfAdmissionThatUseItemid:
    prescript_itemid.append(r[0])

len(res) = 3268
. 0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

# Prescriptions histogram
- y-axis : number of distinct admission that uses particular medicine.
- x-axis : a particular medicine.

In [ ]:
plt.figure(figsize=(20,5))
plt.bar([i for i in range(len(numberOfAdmissionThatUseItemid))],[int(r[1][0]) for r in numberOfAdmissionThatUseItemid])

# save the all the ID for next use

In [ ]:
database = {'input':input_itemid,
            'output':output_itemid,
            'chart':chart_itemid,
            'lab':lab_itemid,
            'microbio':microbio_itemid,
            'prescript':prescript_itemid}
np.save('res/itemids',database);
print "saved !";